In [1]:
#import os
# make sure pyspark tells workers to use python3 not 2 if both are installed
#os.environ['PYSPARK_PYTHON'] = '/home/ubuntu/anaconda3/bin/python3'
#os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/ubuntu/anaconda3/bin/ipython'
#!pip install metapy

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors
import pyspark
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np

In [3]:
#Al ejecutar esta celda se demora un poco, así que un poco de paciencia
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local', "news_cleaned_topic_model") 
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

In [4]:
dfNews=spark.read.csv("file:///home/ubuntu/Caso_estudio/datasets/mini_news.csv", inferSchema=True, header=True, encoding="UTF-8")
dfNews.show(10)


+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
| id|id_news|               title|   publication|              author|      date|  year|month| url|             content|
+---+-------+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
|  0|17283.0|House Republicans...|New York Times|          Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON  —   C...|
|  4|17287.0|Kim Jong-un Says ...|New York Times|       Choe Sang-Hun|2017-01-02|2017.0|  1.0|null|SEOUL, South Kore...|
|  5|17288.0|Sick With a Cold,...|New York Times|         Sewell Chan|2017-01-02|2017.0|  1.0|null|LONDON  —   Queen...|
|  6|17289.0|Taiwan’s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0|  1.0|null|BEIJING  —   Pres...|
|  9|17292.0|Calling on Angels...|New York Times|         Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|
| 10|17293.0|Weak Federal Powe..

# Limpieza del dataset

In [5]:
dfNewsFiltered=dfNews.na.drop(subset=("title","publication","author","year", "date", "month","content"))

In [6]:
dfMainCols= dfNewsFiltered.select('id_news','title','publication','content')
dfMainCols.show(10)

+-------+--------------------+--------------+--------------------+
|id_news|               title|   publication|             content|
+-------+--------------------+--------------+--------------------+
|17283.0|House Republicans...|New York Times|WASHINGTON  —   C...|
|17287.0|Kim Jong-un Says ...|New York Times|SEOUL, South Kore...|
|17288.0|Sick With a Cold,...|New York Times|LONDON  —   Queen...|
|17289.0|Taiwan’s Presiden...|New York Times|BEIJING  —   Pres...|
|17292.0|Calling on Angels...|New York Times|Angels are everyw...|
|17293.0|Weak Federal Powe...|New York Times|With Donald J. Tr...|
|17295.0|Mar-a-Lago, the F...|New York Times|WEST PALM BEACH, ...|
|17296.0|How to form healt...|New York Times|This article is p...|
|17302.0|Suicide Bombing i...|New York Times|BAGHDAD  —   A su...|
|17303.0|Fecal Pollution T...|New York Times|SYDNEY, Australia...|
+-------+--------------------+--------------+--------------------+
only showing top 10 rows



# Modelamiento de Tópicos (LDA)

In [7]:
import nltk
import pandas as pd
import numpy as np
#import codecs
#import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer

nltk.download(['stopwords'])
from nltk.corpus import stopwords
import pyspark.sql.functions as F

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Conteo de las palabras iniciales que están en el título y del contenido de la noticia
def countWords(record):
    textTitle  = record[1]
    textContent = record[3]
    textCombined = textTitle + " " + textContent
    words = textCombined.split()
    longitudTexto=len(words)
    return longitudTexto

udf_countWords = udf(countWords, IntegerType())
dfcountWords = dfMainCols.withColumn("countWords", udf_countWords(struct([dfMainCols[x] for x in dfMainCols.columns])))

In [9]:
from pyspark.ml.feature import  IDF, Tokenizer, CountVectorizer
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA, BisectingKMeans
#from pyspark.sql.functions import monotonically_increasing_id
import re

In [11]:
# Default list of Stopwords

stopWords1 = stopwords.words('english')

stopwords2 = [word.lower() for word in stopWords1] 

stopwords_custom = ['New', 'York','Times','Breitbart', 'News', 'Daily', 'NPR','Reuters','Guardian','CNN','Atlantic','Vox']
stopwords3 = stopwords2 + stopwords_custom

#Stemmer
ps = PorterStemmer() 

def cleanup_text(record):
    textTitle  = str(record[1])
    textPublication=str(record[2])
    textContent = str(record[3])
    
    #setStopwordsCustom =set(stopwords_custom)
    #setStopwordsCustom =setStopwordsCustom.union(set(textPublication.split()))
    #stopwords_custom1 = list(setStopwordsCustom)
    #stopwords_custom= stopwords_custom1
    #stopwords = stopwords_core + stopwords_custom1
    #title=textTitle
    #longPublication =len(textPublication) # Calcular la longitud de la cadena de caracteres  del publisher de la noticia
    #longTitle=len(textTitle)  # Calcular la longitud de la cadena del titulo de la noticia
    
    #if longTitle > longPublication:
    #    textPublisher=textTitle[longTitle-longPublication-1:longTitle] # Extrar caracteres derechos del titulo
    #    if textPublisher==textPublication:
    #        title=textTitle[0:longTitle-longPublication-1]
    #    else:
    #        title=textTitle
    
    textCombined = textTitle + " " + textContent
    # Remove special characters
    textCombined= re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)\n','',textCombined)
    textCombined= re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)',' ',textCombined)
    textCombined= re.sub('[^(a-zA-Z)]',' ',textCombined)
    textCombined= re.sub(' +',' ',textCombined)
    words = textCombined.split()
    tokens3 = [word.lower() for word in words if len(word)>3 and word.lower() not in stopwords3] 
    text_out = [ps.stem(w) for w in tokens3]
    
    #text_out = [re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)\n','',word) for word in words]
    #text_out = [re.sub('(-|-|\u2212|\u2012|\u2013|\u2014|\u2015)',' ',word) for word in words]
    #text_out = [re.sub('[^(a-zA-Z)]',' ',word) for word in words]                                   
    #text_out = [re.sub(' +',' ',word) for word in words] 
    # Remove stopwords and words under X length
    return text_out

In [12]:
udf_cleantext = udf(cleanup_text , ArrayType(StringType()))
dfCombined = dfMainCols.withColumn("CombinedTokens", udf_cleantext(struct([dfMainCols[x] for x in dfMainCols.columns])))

In [13]:
dfCombined.show(10)

+-------+--------------------+--------------+--------------------+--------------------+
|id_news|               title|   publication|             content|      CombinedTokens|
+-------+--------------------+--------------+--------------------+--------------------+
|17283.0|House Republicans...|New York Times|WASHINGTON  —   C...|[hous, republican...|
|17287.0|Kim Jong-un Says ...|New York Times|SEOUL, South Kore...|[jong, say, north...|
|17288.0|Sick With a Cold,...|New York Times|LONDON  —   Queen...|[sick, cold, quee...|
|17289.0|Taiwan’s Presiden...|New York Times|BEIJING  —   Pres...|[taiwan, presid, ...|
|17292.0|Calling on Angels...|New York Times|Angels are everyw...|[call, angel, end...|
|17293.0|Weak Federal Powe...|New York Times|With Donald J. Tr...|[weak, feder, pow...|
|17295.0|Mar-a-Lago, the F...|New York Times|WEST PALM BEACH, ...|[lago, futur, win...|
|17296.0|How to form healt...|New York Times|This article is p...|[form, healthi, h...|
|17302.0|Suicide Bombing i...|Ne

In [14]:
# Term Frequency Vectorization: 
cvTokensCombined = CountVectorizer(inputCol="CombinedTokens", outputCol="rawFeatures", vocabSize = 5000)
cvmodel = cvTokensCombined.fit(dfCombined)
featurizedData = cvmodel.transform(dfCombined)

vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)

#Calculating IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [15]:
rescaledData.show(10)

+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|id_news|               title|   publication|             content|      CombinedTokens|         rawFeatures|            features|
+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|17283.0|House Republicans...|New York Times|WASHINGTON  —   C...|[hous, republican...|(5000,[0,1,2,3,4,...|(5000,[0,1,2,3,4,...|
|17287.0|Kim Jong-un Says ...|New York Times|SEOUL, South Kore...|[jong, say, north...|(5000,[0,1,2,3,4,...|(5000,[0,1,2,3,4,...|
|17288.0|Sick With a Cold,...|New York Times|LONDON  —   Queen...|[sick, cold, quee...|(5000,[0,3,7,11,1...|(5000,[0,3,7,11,1...|
|17289.0|Taiwan’s Presiden...|New York Times|BEIJING  —   Pres...|[taiwan, presid, ...|(5000,[0,1,2,3,4,...|(5000,[0,1,2,3,4,...|
|17292.0|Calling on Angels...|New York Times|Angels are everyw...|[call, angel, end...|(50

In [16]:
# Generate 10 Data-Driven Topics:
lda = LDA(k=10, seed=123, optimizer="em", featuresCol="features")

ldamodel = lda.fit(rescaledData)

#model.isDistributed()
#model.vocabSize()

ldatopics = ldamodel.describeTopics()
ldatopics.show(3)

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[1, 64, 737, 340,...|[0.00537014935798...|
|    1|[140, 806, 736, 1...|[0.00476260288770...|
|    2|[226, 409, 602, 1...|[0.00478580017425...|
+-----+--------------------+--------------------+
only showing top 3 rows



In [17]:
featurizedData.take(10)

[Row(id_news=17283.0, title='House Republicans Fret About Winning Their Health Care Suit - The New York Times', publication='New York Times', content='WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find thems

In [18]:
def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(20,False)

+-----+--------------------------------------------------------------------------------------+
|topic|topic_desc                                                                            |
+-----+--------------------------------------------------------------------------------------+
|0    |[trump, parti, bannon, trade, presid, european, clinton, elect, germani, climat]      |
|1    |[texa, cartel, mexican, zika, mexico, clinton, espn, sport, video, film]              |
|2    |[militari, syria, syrian, forc, russia, islam, govern, iraq, attack, unit]            |
|3    |[trump, health, republican, abort, clinton, ryan, voter, obamacar, vote, plan]        |
|4    |[israel, islam, muslim, attack, isra, palestinian, terror, terrorist, iran, jerusalem]|
|5    |[immigr, trump, comey, depart, obama, texa, investig, hous, illeg, administr]         |
|6    |[castro, park, cuban, french, cuba, china, trump, facebook, marijuana, clinton]       |
|7    |[polic, rubio, student, girl, campu, trump,

In [19]:
ldaResults = ldamodel.transform(rescaledData)

ldaResults.select('id_news','title','publication','content','CombinedTokens','features','topicDistribution').show()

+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|id_news|               title|   publication|             content|      CombinedTokens|            features|   topicDistribution|
+-------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+
|17283.0|House Republicans...|New York Times|WASHINGTON  —   C...|[hous, republican...|(5000,[0,1,2,3,4,...|[0.02579954627314...|
|17287.0|Kim Jong-un Says ...|New York Times|SEOUL, South Kore...|[jong, say, north...|(5000,[0,1,2,3,4,...|[0.01507972754767...|
|17288.0|Sick With a Cold,...|New York Times|LONDON  —   Queen...|[sick, cold, quee...|(5000,[0,3,7,11,1...|[0.04690471816115...|
|17289.0|Taiwan’s Presiden...|New York Times|BEIJING  —   Pres...|[taiwan, presid, ...|(5000,[0,1,2,3,4,...|[0.02324930874689...|
|17292.0|Calling on Angels...|New York Times|Angels are everyw...|[call, angel, end...|(50

In [20]:
ldaResults.select('id_news','topicDistribution').show()

+-------+--------------------+
|id_news|   topicDistribution|
+-------+--------------------+
|17283.0|[0.02579954627314...|
|17287.0|[0.01507972754767...|
|17288.0|[0.04690471816115...|
|17289.0|[0.02324930874689...|
|17292.0|[0.01867067288017...|
|17293.0|[0.23877565687372...|
|17295.0|[0.22378373978935...|
|17296.0|[0.03038324135979...|
|17302.0|[0.01315333213796...|
|17303.0|[0.11992705116337...|
|17305.0|[0.03495147543725...|
|17306.0|[0.05732180516026...|
|17307.0|[0.02403567699286...|
|17311.0|[0.08148541381996...|
|17312.0|[0.05529596176880...|
|17314.0|[0.01423167760756...|
|17318.0|[0.07733065556388...|
|17319.0|[0.03518996867518...|
|17321.0|[0.04231893827890...|
|17323.0|[0.04093512039314...|
+-------+--------------------+
only showing top 20 rows



In [22]:
def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])
    
    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))
ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))
ldatopics_mapped.select(ldatopics_mapped.topic, ldatopics_mapped.topic_desc).show(20,False)

+-----+--------------------------------------------------------------------------------------+
|topic|topic_desc                                                                            |
+-----+--------------------------------------------------------------------------------------+
|0    |[trump, parti, bannon, trade, presid, european, clinton, elect, germani, climat]      |
|1    |[texa, cartel, mexican, zika, mexico, clinton, espn, sport, video, film]              |
|2    |[militari, syria, syrian, forc, russia, islam, govern, iraq, attack, unit]            |
|3    |[trump, health, republican, abort, clinton, ryan, voter, obamacar, vote, plan]        |
|4    |[israel, islam, muslim, attack, isra, palestinian, terror, terrorist, iran, jerusalem]|
|5    |[immigr, trump, comey, depart, obama, texa, investig, hous, illeg, administr]         |
|6    |[castro, park, cuban, french, cuba, china, trump, facebook, marijuana, clinton]       |
|7    |[polic, rubio, student, girl, campu, trump,

In [23]:
from pyspark.sql.types import IntegerType


def asignTopic(topicDistribution):
    topic = topicDistribution[0]
    index_topic = 0
    for index in range(len(topicDistribution)):
        if (topicDistribution[index]>topic):
            topic=topicDistribution[index]
            index_topic=index
    return index_topic

udf_asignTopic = udf(asignTopic, IntegerType())
dfNewsTopic = ldaResults.withColumn("mainTopic", udf_asignTopic(ldaResults.topicDistribution))

In [24]:
dfNewsTopic.select('mainTopic').show()

+---------+
|mainTopic|
+---------+
|        3|
|        2|
|        6|
|        6|
|        6|
|        9|
|        0|
|        8|
|        2|
|        8|
|        7|
|        1|
|        2|
|        3|
|        2|
|        4|
|        6|
|        4|
|        6|
|        8|
+---------+
only showing top 20 rows

